In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# read file names from file_list.csv
file_list = pd.read_csv("file_list.csv")
csv_files = [filename + '.csv' for filename in file_list['file_name']]

# get the range of index values
path = r"/home/tzahid/Desktop/dataset20"
max_index = 0
for file in csv_files:
    data = pd.read_csv(os.path.join(path, file), usecols=[0])
    max_index = max(max_index, data['index_date'].max())

index_range = range(1, max_index+1)

# create an empty dataframe with the desired index
df = pd.DataFrame(index=index_range)

# loop through the csv files and extract the 5th column and add it to the dataframe
for file in tqdm(csv_files, desc='Processing CSV files'):
    col_name = os.path.splitext(file)[0]
    col_name = col_name.replace(" ", "_")  # replace spaces with underscores in column names
    data = pd.read_csv(os.path.join(path, file), usecols=[0, 4], header=0, skiprows=[0], names=['index_date', col_name]) # modify usecols to define which columns to extract, here 0 and 4
    data = data.drop_duplicates(subset=['index_date'])  # drop any duplicate rows
    data = data.set_index('index_date')  # set the index to the 'index_date' column
    data = data.reindex(index_range, fill_value=0)  # add missing index rows and fill with 0
    df[col_name] = data[col_name]  # add the column to the main dataframe

# save the concatenated dataframe to a new csv file
df.to_csv(os.path.join(path, 'concatenated_sensor_values.csv'), index_label='index_date')

# print the range of index values
print(f"Index range: 1 - {max_index}")

# print final message
print(f"Concatenation complete. {len(csv_files)} files concatenated. Output saved to 'concatenated_sensor_values.csv'.")
